In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
aapl_text_direction_date.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv
lstm_amzn_fulltext/
text_kfold_aapl.csv


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_amzn = pd.read_csv('aapl_text_direction_date.csv')


In [ ]:
df_amzn.dropna(inplace=True)

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
#title_amzn=df_amzn['title'].tolist()
text_amzn=df_amzn['processedText'].to_numpy()
label_amzn=df_amzn['direction'].to_numpy()


#print(title_amzn[0])
print(text_amzn[0])
print(label_amzn[0])

september 2 2018 trading ideas comments evidence apple next big product pair computer glasses aapl apple said earlier week bought small colorado company worked augmented reality glasses it pretty clear point apple working pair course apple working new products 5 billion headquarters research labs
1


In [ ]:
class_label_amzn = (label_amzn + 1) / 2
#class_label_amzn = pd.get_dummies(label_amzn).values

In [ ]:
encoder = tf.keras.layers.TextVectorization(standardize=None)
encoder.adapt(text_amzn)

In [ ]:

print(encoder(text_amzn[:3]).numpy())

[[ 168  144   29 ...    0    0    0]
 [ 622 3647  567 ...    0    0    0]
 [ 124  436  963 ...    4  822  124]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_amzn, class_label_amzn, test_size=0.2, random_state=56)

In [ ]:
X_train.shape

(7997,)

In [ ]:
Y_train.shape

(7997,)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Embedding
from keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
layers = [
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
]
model = tf.keras.models.Sequential(layers)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
model.fit(X_train[:1000], Y_train[:1000], epochs=10, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/10
13/13 [==============================] - 188s 14s/step - loss: 0.6935 - accuracy: 0.5337 - val_loss: 0.6934 - val_accuracy: 0.4600
Epoch 2/10
13/13 [==============================] - 171s 13s/step - loss: 0.6919 - accuracy: 0.5337 - val_loss: 0.6936 - val_accuracy: 0.4600
Epoch 3/10
13/13 [==============================] - 171s 14s/step - loss: 0.6906 - accuracy: 0.5337 - val_loss: 0.6940 - val_accuracy: 0.4600
Epoch 4/10
13/13 [==============================] - 172s 14s/step - loss: 0.6889 - accuracy: 0.5337 - val_loss: 0.6943 - val_accuracy: 0.4600
Epoch 5/10
13/13 [==============================] - 173s 14s/step - loss: 0.6869 - accuracy: 0.5337 - val_loss: 0.6949 - val_accuracy: 0.4600
Epoch 6/10
13/13 [==============================] - 170s 14s/step - loss: 0.6840 - accuracy: 0.5337 - val_loss: 0.6952 - val_accuracy: 0.4600
Epoch 7/10
13/13 [==============================] - 173s 14s/step - loss: 0.6800 - accuracy: 0.5337 - val_loss: 0.6963 - val_accuracy: 0.4600
Epoch 

In [ ]:
model.fit(X_train[:1000], Y_train[:1000], epochs=10, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/10
13/13 [==============================] - 168s 13s/step - loss: 0.5865 - accuracy: 0.5375 - val_loss: 0.6816 - val_accuracy: 0.4600
Epoch 2/10
13/13 [==============================] - 169s 13s/step - loss: 0.5541 - accuracy: 0.5537 - val_loss: 0.6938 - val_accuracy: 0.4650
Epoch 3/10
13/13 [==============================] - 179s 14s/step - loss: 0.4940 - accuracy: 0.7225 - val_loss: 0.6810 - val_accuracy: 0.5100
Epoch 4/10
13/13 [==============================] - 170s 14s/step - loss: 0.4135 - accuracy: 0.8062 - val_loss: 0.7716 - val_accuracy: 0.5300
Epoch 5/10
13/13 [==============================] - 177s 14s/step - loss: 0.3135 - accuracy: 0.8963 - val_loss: 0.7870 - val_accuracy: 0.5200
Epoch 6/10
13/13 [==============================] - 162s 13s/step - loss: 0.2267 - accuracy: 0.9325 - val_loss: 1.0592 - val_accuracy: 0.5150
Epoch 7/10
13/13 [==============================] - 173s 11s/step - loss: 0.1935 - accuracy: 0.9375 - val_loss: 0.8993 - val_accuracy: 0.5450
Epoch 

In [ ]:
eva = model.evaluate(X_test, Y_test, verbose=2)

63/63 - 113s - loss: 1.2100 - accuracy: 0.5205 - 113s/epoch - 2s/step
